In [1]:
from timeseries_fastai.imports import *
from timeseries_fastai.data import *
from timeseries_fastai.core import *
from timeseries_fastai.models import *
from timeseries_fastai.tabular import *

from ucr import *

In [2]:
PATH = get_ucr()

In [3]:
PATH.ls()

(#135) [Path('/home/doug/.fastai/data/Univariate2018_arff/Yoga'),Path('/home/doug/.fastai/data/Univariate2018_arff/SummaryData.csv'),Path('/home/doug/.fastai/data/Univariate2018_arff/Strawberry'),Path('/home/doug/.fastai/data/Univariate2018_arff/Chinatown'),Path('/home/doug/.fastai/data/Univariate2018_arff/SemgHandGenderCh2'),Path('/home/doug/.fastai/data/Univariate2018_arff/Symbols'),Path('/home/doug/.fastai/data/Univariate2018_arff/GunPointMaleVersusFemale'),Path('/home/doug/.fastai/data/Univariate2018_arff/MiddlePhalanxOutlineAgeGroup'),Path('/home/doug/.fastai/data/Univariate2018_arff/Herring'),Path('/home/doug/.fastai/data/Univariate2018_arff/InsectEPGRegularTrain')...]

In [4]:
NON_NAN_TASKS[0:5]

['ACSF1', 'Adiac', 'ArrowHead', 'BME', 'Beef']

In [5]:
def max_bs(N):
    N = N//6
    k=1
    while (N//2**k)>1: k+=1
    return min(2**k, 32)

In [6]:
def get_dls(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    return TSDataLoaders.from_dfs(df_train, df_test, x_cols=x_cols, label_col='target', 
                                  bs=bs, num_workers=workers)

In [7]:
dls = get_dls(PATH, 'Adiac')

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/Adiac


In [8]:
def get_dls2(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    
    df_main = stack_train_valid(df_train, df_test)
    splits=[range_of(df_train), list(range(len(df_train), len(df_main)))]
    to = TSPandas(df_main, [Normalize], x_names=x_cols, y_names='target', splits=splits)
    
    return to.dataloaders(bs, 2*bs)

In [9]:
dls = get_dls2(PATH, 'Adiac')

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/Adiac


In [10]:
dls.c

37

In [11]:
x,y = dls.train.one_batch()

In [12]:
x.shape

torch.Size([32, 1, 176])

In [13]:
def get_model(dls, arch):
    num_classes = dls.c
    arch = arch.lower()
    if arch=='resnet':     model = create_resnet(1, num_classes, conv_sizes=[64, 128, 256])
    elif arch=='fcn':      model = create_fcn(1, num_classes, ks=9, conv_sizes=[128, 256, 128])
    elif arch=='mlp':      model = create_mlp(dls.train.one_batch()[0].shape[-1], num_classes)
    elif arch=='inception':model = create_inception(1, num_classes)
    else: 
        print('Please chosse a model in [resnet, FCN, MLP, inception]')
        return None
    return model

In [14]:
LabelSmoothingCrossEntropy??

In [15]:
def train_task(path, task='Adiac', arch='resnet', epochs=40, lr=5e-4):
    "trains arch over task with params"
    dls = get_dls2(path, task)
    model = get_model(dls, arch)
    cbs =  MixUp(0.2)
    learn = Learner(dls, model=get_model(dls, arch), opt_func=Adam, \
            metrics=[accuracy])
    learn.fit_one_cycle(epochs, lr, cbs=cbs)                          
    return learn.to_fp16()

In [16]:
learn = train_task(PATH, 'Adiac', arch='mlp', epochs=4)

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.636275,3.431992,0.135550,00:00
1,3.212873,2.779463,0.347826,00:00
2,2.883641,2.355886,0.388747,00:00
3,2.706255,2.220112,0.414322,00:00


In [17]:
def compute_metrics(learn):
    "compute oguiza Metrics on UCR"
    results = np.array(learn.recorder.values)
    acc_ = results[-1,-1]
    accmax_ = results[:, -1].max()
    loss_ = results[-1,0]
    val_loss_ = results[-1,1]
    return acc_, accmax_, loss_, val_loss_ 

In [18]:
learn = train_task(PATH, NON_NAN_TASKS[0], epochs=4)

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,2.289219,2.046521,0.230000,00:00
1,1.833374,1.634123,0.530000,00:00
2,1.597813,1.330528,0.610000,00:00
3,1.450870,1.088530,0.670000,00:00


In [19]:
def run_tasks(tasks, arch='resnet', lr=1e-3, epochs=1):
    results = [compute_metrics(train_task(PATH, task, arch, epochs, lr)) for task in tasks]
    return pd.DataFrame(data=results, columns=['acc', 'acc_max', 'train_loss', 'val_loss'], index=tasks)

In [20]:
run_tasks(NON_NAN_TASKS)

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,2.022443,1.900637,0.430000,00:00


Loading files from: /home/doug/.fastai/data/Univariate2018_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.335272,3.320352,0.074169,00:01


Loading files from: /home/doug/.fastai/data/Univariate2018_arff/ArrowHead


...

Loading files from: /home/doug/.fastai/data/Univariate2018_arff/WormsTwoClass


epoch,train_loss,valid_loss,accuracy,time
0,0.765072,0.708153,0.571429,00:00


Loading files from: /home/doug/.fastai/data/Univariate2018_arff/Yoga


epoch,train_loss,valid_loss,accuracy,time
0,0.735078,0.683407,0.539333,00:00


,acc,acc_max,train_loss,val_loss
ACSF1,0.430000,0.430000,2.022443,1.900637
Adiac,0.074169,0.074169,3.335272,3.320352
ArrowHead,0.491429,0.491429,1.013500,1.328084
BME,0.326667,0.326667,1.288823,1.293374
Beef,0.400000,0.400000,1.736621,1.423172
...,...,...,...,...
Wine,0.500000,0.500000,0.638970,0.826242
WordSynonyms,0.139498,0.139498,2.943024,3.235155
Worms,0.272727,0.272727,1.633147,1.500055
WormsTwoClass,0.571429,0.571429,0.765072,0.708153
